# Conflict Volatility (fatalities categories)


This notebook creates a way to observe countries' levels of violence and their likelihood of transitioning to various levels of violence over time periods. 

This notebook will create a new categorical variable which converts the actuals and predictions into the following groups: (0) Non-violent; (1) Low-violence; (2) Violent; (3) High-violence; (4) Extreme Violence.

**Non-violent** refers to zero conflict-related deaths in a given country for each month over the past 3-months.

**Low-violence** refers to 1-9 conflict-related deaths in a given country for each month over the past 3-months.

**Violent** refers to 9-99 or more conflict-related deaths in a given country for each month over the past 3-months.

**High-violence** refers to 100-1000 or more conflict-related deaths in a given country for each month over the past 3-months.

**Extreme violence** refers to 1000 or more conflict-related deaths in a given country for each month over the past 3-months.

### Configuration and Packages

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
import views_mapper2
from views_mapper2.mapper2 import Mapper2
from views_mapper2 import color
from views_mapper2.label_writer import vid2date
from views_mapper2.dictionary_writer import standard_scale

# Packages from Mapper2
from ingester3.ViewsMonth import ViewsMonth
from ingester3.Country import Country
import pycountry
from datetime import date
import os
import numpy as np

# Mapper
import geopandas as gpd

import sqlalchemy as sa
#from ingester3.config import source_db_path

# Other packages
import pickle as pkl

#Parallelization
from joblib import Parallel, delayed, cpu_count
from functools import partial

from pathlib import Path

# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')
import os
from pathlib import Path

#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

#from FetchData import FetchData, RetrieveFromList, ReturnQsList, get_df_from_datasets_by_name
#from ViewsEstimators import *

In [ ]:
dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
prod_id = '2022_04_t01'
level = 'cm'
months_prior = 3


#note the home path has been automated with the os.path.expanduser function
#no need to adjust the folder as long as ViEWS on the dropbox is the destination

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS_new/'
Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/'
localgitpath = home + '/Desktop/VIEWS_new/'


##################
print('OS Login is:', os.getlogin())
print('OS path is set to:', home)
print('Local Git path is set to:', localgitpath)
print('Dropbox path is set to:', Mydropbox)

## 1: Fetching and merge the data


    1: Pull in the actuals, restrict the data to (EndOfHistory-40) to EndOfHistory

    2: Pull in the future categorical future predictions, steps 1 to 36 months into the future
    
    3: Merge the DFs: create a single df with actuals and predictions

### 1.1: Fetch acutals

The following cell calls fatalities per UCDP GED 

In [ ]:
qs1 = (Queryset("actuals_monthly", "country_month")
            .with_column(Column("ged_best_sb", from_table= "ged2_cm", from_column="ged_sb_best_sum_nokgi")
                        .transform.missing.fill())   
            .with_theme("fatalities")
            .describe(""" Fatalities conflict history, cm level Monthly Update"""))
actuals = qs1.publish().fetch()

actuals = actuals.loc[EndOfHistory-40:EndOfHistory]
actuals = actuals.reset_index()
actuals['step'] = actuals['month_id'] - EndOfHistory
actuals = actuals.set_index(['step', 'country_id'], drop = True)
actuals.rename(columns={'ged_best_sb':'fatalities'}, inplace=True)

### 1.2: Fetch the future predictions with their categorical uncertainty values

Presently, this simply reads in the .csv file; however, I would like it to read from pred_store. It also transforms 'steps_combined' into the exponential form. 

In [ ]:
future = pd.read_csv('Categorical_probabilities.csv')

future['step'] = future['month_id'] - EndOfHistory
future = future.set_index(['step', 'country_id'], drop = True)
future['step_combined'] = np.expm1(future['step_combined'])
future.rename(columns={'step_combined':'fatalities'}, inplace=True)

### 1.3: Merge the data


In [ ]:
merge = pd.concat([actuals, future], axis=0)

## 2: Creating Fatalities Categories (Fat_Cat)

This section creates fat_cat. Here, I write the code the defines the categories. 

# 2.1: Defining the categories

This cell defines a function that creates a multiclass categorical variable grouping fatalities 
    
    0: Peaceful = a country-month without any fatalities
    1: Low-violence = a country-month with 1-9 fatalities
    2: Violent = a country-month with more than 9-99 fatalities
    3: High-violence = a country-month with more than 100-999 fatalities
    4: Extreme violence = a country-month with more than 1000 fatalites
    

In [ ]:
fatcat = merge.copy()
fatcat = fatcat.reset_index(drop=False)

In [ ]:
def ged_fatalities(x):
    if x < 0.5:
        return 0
    elif x < 10:
        return 1
    elif x < 100:
        return 2
    elif x < 1000:
        return 3
    else:
        return 4
       
def ged_intensity_lvl(x):
    if x < 0.5:
        return 0
    elif x < 100:
        return 1
    else:
        return 2  

In [ ]:
fatcat['fatcat'] = fatcat['fatalities'].apply(ged_fatalities)

# Create a variable that subtracts the observation in the row above by the row below and prints the output
fatcat['fatcat_change'] = fatcat.groupby('country_id')['fatcat'].diff()


# Fill the top values of ['status'] with zeros
fatcat['fatcat_change'] = fatcat['fatcat_change'].fillna(0)

fatcat = fatcat.set_index(['step','country_id'], drop = True)


In [ ]:
# This cell creates 'fatcat_nominal'

fatcat['fatcat_nominal'] = fatcat['fatcat']
fatcat = fatcat.reset_index()

fat_classes = [
    (fatcat['fatcat'] == 0), 
    (fatcat['fatcat'] == 1),
    (fatcat['fatcat'] == 2),
    (fatcat['fatcat'] == 3),
    (fatcat['fatcat'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violence', 'High-violence', 'Extreme Violence']

fatcat['fatcat_nominal'] = np.select(fat_classes, fat_categories)

fatcat = fatcat.set_index(['step','country_id'], drop = True)

In [ ]:
# This cell creates 'monthly_change' variable

fatcat['monthly_change'] = fatcat['fatcat_change']
fatcat = fatcat.reset_index()

violence_conditions = [
    (fatcat['monthly_change'] == -2), 
    (fatcat['monthly_change'] == -1),
    (fatcat['monthly_change'] == 0),
    (fatcat['monthly_change'] == 1),
    (fatcat['monthly_change'] == 2)
]

violence_categories = ['Rapid Descalation', 'De-escalation', 'Status Quo', 'Escalation', 'Rapid Escalation']

fatcat['monthly_change'] = np.select(violence_conditions, violence_categories)

In [ ]:
#fatcat['monthly_change'] = np.select(violence_conditions, violence_categories)

# This creates a rolling mean of the violence over the past 'months prior' parameter
fatcat['rolling_mean_violence'] = fatcat.groupby('country_id')['fatalities'].transform(lambda x: x.shift(1).rolling(months_prior, 1).mean())

fatcat['intensity_lvl'] = fatcat['rolling_mean_violence'].apply(ged_intensity_lvl)

#Reset the df to only include 12 months prior
fatcat = fatcat.loc[fatcat['step'] >= -12]

#Reset the index
fatcat = fatcat.set_index(['step', 'country_id'], drop = True)


In [ ]:
fatcat

In [ ]:
#fatcat.to_csv('/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/fatcatpeak.csv',sep = ';')

## 3: Creating Change Lists

The following cells shows the countries that are expected to shift between fatcats

### 3.1: Creating a list from Step0 

In [ ]:
fatcat0 = fatcat.reset_index(drop=False)
fatcat0 = fatcat0[fatcat0['step'] == 0] 
fatcat0_pivot = pd.pivot_table(fatcat0, values=['fatalities', 'multi_0_logit','multi_1_logit', 'multi_2_logit', 'multi_3_logit',
                                                'multi_4_logit', 'fatcat', 'fatcat_nominal', 'fatcat_change', 'monthly_change', 'rolling_mean_violence',
                                                'intensity_lvl'],
                               index=['country_id'])

fatcat0_pivot.rename(columns={'fatalities': 'fatalities_step_0'}, inplace=True)
fatcat0_pivot.rename(columns={'fatcat': 'fatcat_step_0'}, inplace=True)

### 3.2: Creating a list from StepX 

In [ ]:
fatcat12 = fatcat.reset_index(drop=False)
fatcat12 = fatcat12[fatcat12['step'] == 12] 
fatcat12_pivot = pd.pivot_table(fatcat12, values=['fatalities', 'multi_0_logit','multi_1_logit', 'multi_2_logit', 'multi_3_logit',
                                                'multi_4_logit', 'fatcat', 'fatcat_change', 'monthly_change', 'rolling_mean_violence',
                                                'intensity_lvl', 'fatcat_nominal'],
                               index=['country_id'])

fatcat12_pivot.rename(columns={'fatalities': 'fatalities_step_12'}, inplace=True)
fatcat12_pivot.rename(columns={'fatcat': 'fatcat_step_12'}, inplace=True)
#fatcat_3 = fatcat_3.sort_values(by=['fatalities'], ascending=False)

### 3.3: Joining these data 

In [ ]:
fatcat_list = fatcat0_pivot.join(fatcat12_pivot, lsuffix='_step_0', rsuffix='_step_12')
fatcat_list = fatcat_list.sort_values('fatalities_step_0', ascending=False)

In [ ]:
fatcat_list['fatcat_change'] = fatcat_list['fatcat_step_12'] - fatcat_list['fatcat_step_0']

violence_conditions2 = [
    (fatcat_list['fatcat_change'] == -2), 
    (fatcat_list['fatcat_change'] == -1),
    (fatcat_list['fatcat_change'] == 0),
    (fatcat_list['fatcat_change'] == 1),
    (fatcat_list['fatcat_change'] == 2)
]

violence_categories = ['Rapid Descalation', 'De-escalation', 'Status Quo', 'Escalation', 'Rapid Escalation']

fatcat_list['facat_change_nominal'] = np.select(violence_conditions2, violence_categories)

### 3.4: Data manipulation

In [ ]:
def cid2name(country_id):
    """
    Function useful for labeling countries if needed by their name
    Create a string output name by country id
    Uses ingester3 Country as basis
    :param country_id: Views Country ID
    :return: name
    """
    name = Country(country_id).name
    return name

In [ ]:
fatcat_list = fatcat_list.reset_index(drop=False)

In [ ]:
c_id = [int(x) for x in fatcat_list['country_id']] 
country_names = []
for x in (c_id):
        country_names.append(cid2name(x)) 

In [ ]:
fatcat_list['country'] = country_names
fatcat_list = fatcat_list.set_index('country_id', drop = True)

#### Creating 'fatcat_nominal' for steps 1 and 12

In [ ]:
# Creates: Fatcat list 0 nominal

fatcat_list['fatcat_0_nominal'] = fatcat_list['fatcat_step_0']
fatcat_list = fatcat_list.reset_index()

fat_classes = [
    (fatcat_list['fatcat_step_0'] == 0), 
    (fatcat_list['fatcat_step_0'] == 1),
    (fatcat_list['fatcat_step_0'] == 2),
    (fatcat_list['fatcat_step_0'] == 3),
    (fatcat_list['fatcat_step_0'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence']

fatcat_list['fatcat_0_nominal'] = np.select(fat_classes, fat_categories)

fatcat_list = fatcat_list.set_index(['country_id'], drop = True)

In [ ]:
# Creates: Fatcat list 12 nominal

fatcat_list['fatcat_12_nominal'] = fatcat_list['fatcat_step_12']
fatcat_list = fatcat_list.reset_index()

fat_classes = [
    (fatcat_list['fatcat_step_12'] == 0), 
    (fatcat_list['fatcat_step_12'] == 1),
    (fatcat_list['fatcat_step_12'] == 2),
    (fatcat_list['fatcat_step_12'] == 3),
    (fatcat_list['fatcat_step_12'] == 4)
]

fat_categories = ['Non-violent', 'Low Violence', 'Violent', 'High-violence', 'Extreme Violence']

fatcat_list['fatcat_12_nominal'] = np.select(fat_classes, fat_categories)

fatcat_list = fatcat_list.set_index(['country_id'], drop = True)

## 4: Fatcat Lists

### 4.1: Escalation List

This list the countries that are expected to esclate in violence from the EndOfHistory until 12-months later

In [ ]:
escalationlist = fatcat_list.copy()
escalationlist = escalationlist.loc[escalationlist['fatcat_change'] >= 1 ] 
escalationlist = escalationlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
escalationlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/escalationlist_12.md')
escalationlist
#print(escalationlist.to_markdown())

### 4.1: De-escalation List

This list the countries that are expected to de-esclate in violence from the EndOfHistory until 12-months later

In [ ]:
deescalationlist = fatcat_list.copy()
deescalationlist = deescalationlist.loc[deescalationlist['fatcat_change'] < 0 ] 
deescalationlist = deescalationlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
deescalationlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/deescalationlist_12.md')
#print(deescalationlist.to_markdown())
deescalationlist

### 4.1: Status Quo List

This list the countries that are expected to de-esclate in violence from the EndOfHistory until 12-months later

In [ ]:
sqlist = fatcat_list.copy()
sqlist = sqlist.loc[sqlist['fatcat_change'] == 0 ] 
sqlist = sqlist.sort_values(by = ['fatalities_step_12'], ascending=[False])
sqlist.to_markdown(buf= '/Users/chandlerwilliams/Desktop/VIEWS_new/test_folder/sqlist_12.md')
#print(sqlist.to_markdown())
sqlist

## FatCat Lists Condensed

This cell creates FatCat lists that are condensed to only include: 

    - Country Name
    - FatCat_change_nominal
    - Fatcat_0_nominal
    - Fatcat_12_nominal
    - Fat0
    - Fat12


In [ ]:
deescalationlist_short = deescalationlist[['country', 'facat_change_nominal', 'fatcat_0_nominal', 'fatcat_12_nominal',
                                           'fatalities_step_0', 'fatalities_step_12','rolling_mean_violence_step_0', 'rolling_mean_violence_step_12']]



In [ ]:
deescalationlist_short

In [ ]:
deescalationlist_short.rename(columns={'country': 'Country','facat_change_nominal': 'Estimated Change in Violence',
                                      'fatcat_0_nominal': 'Present Violence Level','fatcat_12_nominal': 'Predicted Violence Level',
                                      'fatalities_step_0': 'Present Fatalities','rolling_mean_violence_step_0': 'Mean Violence 3-months Prior (present)',
                                      'rolling_mean_violence_step_12': 'Mean Violence 3-months Prior (predicted)','fatalities_step_12': 'Predicted Fatalities'},
                              inplace = True)

In [ ]:
deescalationlist_short
#print(deescalationlist_short.to_markdown())